In [2]:
%load_ext autoreload
%autoreload 2

# Read

In [2]:
import pandas as pd
df = pd.read_parquet("/data/lux70/data/pfam/val.parquet")# df = pd.read_parquet("/data/lux70/data/pfam/val_shard_info.parquet")

In [3]:
df.shape

(8596167, 11)

In [4]:
df = df[~df.GO_counts.isna()]
df.head()

,pfam_id,organism,sequence,GO_id,GO_term,GO_level,GO_counts,GO_idx,organism_index,organism_counts,seq_len
11695,PF15894,9ENTR,QFYQRYFAATKNVSWLARQATAQRLEVLNSLMQWEVTKSASEH,GO:0046325,negative regulation of glucose import,process,1.0,2130,472,85933,43
11696,PF15894,9ENTR,QFWLKYFTATERLSWLACLSAPQRLKMLEELMQWELRD,GO:0046325,negative regulation of glucose import,process,1.0,2130,472,85933,38
11697,PF15894,SALET,QFWLRYFAATEKTSWLACLSAPQRLKMLAELMQWEATD,GO:0046325,negative regulation of glucose import,process,1.0,2130,2465,1513,38
11698,PF15894,CITK8,QFYLKYFTATEGLSWLACPSAPQRLKMLEELMQWEVTA,GO:0046325,negative regulation of glucose import,process,1.0,2130,2456,1399,38
11699,PF15894,9ENTR,RFFQQYFAATQFVCSGWLARLTVEQRLRMLEDLMQWEVTTPMPENAN,GO:0046325,negative regulation of glucose import,process,1.0,2130,472,85933,47


In [5]:
import numpy as np
from plaid.utils import round_to_multiple

tmp = df[(df.GO_term == "DNA-templated transcription initiation")].groupby("pfam_id")['seq_len'].mean()
print(tmp)

pids = tmp.index
half_seqlens = [round_to_multiple(v / 2, 4) for v in tmp.values]

print(half_seqlens)

pfam_id
PF00352     79.276549
PF02291    119.764097
PF03540     48.356998
PF04539     75.140669
PF04542     68.777518
PF04545     50.665917
PF04963    191.626108
Name: seq_len, dtype: float64
[40, 60, 24, 36, 36, 24, 96]


In [7]:
df[(df.GO_term == "RNA binding")].GO_idx

94274      6
94275      6
94276      6
94277      6
94278      6
          ..
7499965    6
7499966    6
7499967    6
7499968    6
7499969    6
Name: GO_idx, Length: 18887, dtype: int64

In [8]:
from plaid.datasets import NUM_FUNCTION_CLASSES, NUM_ORGANISM_CLASSES

organism_idx = NUM_ORGANISM_CLASSES
function_idx = 6
print(organism_idx, function_idx)

3617 6


In [13]:
from omegaconf import OmegaConf
import hydra

cfg = OmegaConf.load("/homefs/home/lux70/code/plaid/configs/pipeline/sample/sample_latent.yaml")

N, L = 8, 72
input_dim = 32
assert L % 4 == 0
shape = (N, L, input_dim)

by_scale = {}

for scale in np.arange(6.5, 10., 0.5):
    cfg['function_idx'] = function_idx
    cfg['organism_idx'] = organism_idx
    cfg['length'] = L
    cfg['num_samples'] = N
    cfg['cond_scale'] = float(scale)
    cfg['sampling_timesteps'] = 500
    
    solver = hydra.utils.instantiate(cfg)
    x = solver.sample()

    by_scale[scale] = x.detach().cpu().numpy()

/homefs/home/lux70/code/plaid/src/plaid/pipeline/_sample.py:165: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(model_path)


sampling loop time step:   0%|          | 0/500 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/500 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/500 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/500 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/500 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/500 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/500 [00:00<?, ?it/s]

In [14]:
by_scale.keys()

dict_keys([6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5])

In [29]:
# import pickle as pkl
# with open("/homefs/home/lux70/code/plaid/by_scale_tmp2.pkl", "wb") as f:
#     pkl.dump(by_scale, f)

# with open("/homefs/home/lux70/code/plaid/by_scale_tmp.pkl", "rb") as f:
#     by_scale_1 = pkl.load(f)

# with open("/homefs/home/lux70/code/plaid/by_scale_tmp2.pkl", "rb") as f:
#     by_scale = pkl.load(f)

# results = by_scale | by_scale_1

In [34]:
results.keys()

dict_keys([6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0])

# Decode

In [18]:
from cheap.pretrained import CHEAP_pfam_shorten_2_dim_32 
cheap_model = CHEAP_pfam_shorten_2_dim_32()

import torch
device = torch.device("cuda")
_ = cheap_model.to(device)

from cheap.proteins import LatentToSequence,LatentToStructure
latent_to_sequence = LatentToSequence()
latent_to_sequence.to(device)

latent_to_structure = LatentToStructure()
latent_to_structure.to(device)

Using tanh layer at bottleneck...
Finished loading HPCT model with shorten factor 2 and 32 channel dimensions.
Creating ESMFold embedding only model...
ESMFold embedding only model created in 29.18 seconds


/homefs/home/lux70/code/cheap-proteins/src/cheap/decoder.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path)
Creating ESMFold...
ESMFold model 

In [37]:
cur_cond_scale = 9.5
sampled_latent = results[cur_cond_scale]

print(sampled_latent.shape)
final_sample = sampled_latent[:, -1, :, :]
print(final_sample.shape)
print(final_sample.max(), final_sample.min())

final_sample = torch.from_numpy(final_sample).to(device)
sampled_uncompressed = cheap_model.decode(final_sample)
print(sampled_uncompressed.min(), sampled_uncompressed.max())

from cheap.utils import LatentScaler
latent_scaler = LatentScaler()
sampled_unscaled = latent_scaler.unscale(sampled_uncompressed) 

sequences = latent_to_sequence.to_sequence(sampled_unscaled)[-1]
print(sequences[:3])

pdb_strs, raw_outputs = latent_to_structure.to_structure(sampled_unscaled, return_raw_outputs=True, sequences=sequences, batch_size=32, num_recycles=1)


import py3Dmol

# for i in range(len(pdb_strs)):
# for i in range(10,20): 
# for i in range(0, 10): 
for i in range(30): 
    view = py3Dmol.view(width=600, height=600)
    view.addModelsAsFrames(pdb_strs[i])
    
    # Apply the plDDT color scheme
    # view.setStyle({'cartoon': {'color': {'prop': 'b', 'gradient': 'roygb', 'min': 0, 'max': 100}}})
    view.setStyle({'cartoon': {'color': {'prop': 'b', 'gradient': 'roygb', 'min': 50, 'max': 90}}})
    
    # # Add surface representation with plDDT-based color
    view.addSurface(py3Dmol.VDW, {'opacity': 0.7, 'colorscheme': {'prop': 'b', 'gradient': 'roygb', 'min': 50, 'max': 90}})
    # view.addSurface(py3Dmol.VDW, {'opacity': 0.7, 'colorscheme': {'prop': 'b', 'gradient': 'roygb', 'min': 0, 'max': 100}})

    view.zoomTo()
    view.show()

(8, 501, 72, 32)
(8, 72, 32)
1.0097377 -0.99580336
tensor(-835.9632, device='cuda:0') tensor(2791.8604, device='cuda:0')
['AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA', 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA', 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA']


(Generating structure): 100%|████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.87s/it]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

...

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

IndexError: list index out of range